In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [3]:
mnist = tf.keras.datasets.mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [4]:
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2552 - accuracy: 0.9270 - val_loss: 0.1353 - val_accuracy: 0.9600
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1147 - accuracy: 0.9665 - val_loss: 0.1004 - val_accuracy: 0.9676
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0785 - accuracy: 0.9771 - val_loss: 0.0842 - val_accuracy: 0.9742
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0592 - accuracy: 0.9822 - val_loss: 0.0774 - val_accuracy: 0.9759
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0470 - accuracy: 0.9850 - val_loss: 0.0789 - val_accuracy: 0.9773
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0370 - accuracy: 0.9883 - val_loss: 0.0797 - val_accuracy: 0.9760
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0290 - accuracy: 0.9912 - val_loss: 0.0880 - val_accuracy:

In [5]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0820 - accuracy: 0.9782


In [6]:
n = 1
old = model.get_weights()

In [7]:
def remove_random_nodes(acc, loss, weights, n, to_test, x_train, y_train, v=0, remove='positive'):
    new_loss = loss
    new_acc = acc
    if remove == 'positive':
        best_score = 0
    elif remove == 'always':
        best_score = -1e20
    else:
        best_score = 0.5
    best_model = copy.deepcopy(weights)
    nodes_removed = np.array([])
    for _ in tqdm.trange(to_test):   
        new = copy.deepcopy(weights)
        to_drop = np.random.choice(len(new[1]), n, replace=False)
        for i in to_drop:
            new[0][:,i] = 0
            new[1][i] = 0
            new[2][i,:] = 0
        model.set_weights(new)
        new_loss, new_acc = model.evaluate(x_train, y_train, verbose=v)
        score = (1 - (new_loss / loss)) + ((new_acc / acc) - 1)
        if best_score < score:
            best_score = score
            best_model = copy.deepcopy(new)
            nodes_removed = to_drop.copy()
    return best_model, best_score, nodes_removed

In [8]:
to_check = 100
new_loss = loss
new_acc = acc
best_score = 0
best_model = copy.deepcopy(old)

for i in tqdm.trange(to_check):   
    new = copy.deepcopy(old)
    #for i in range(len(old)):
    #    new[i] = old[i].copy()
    to_drop = np.random.choice(len(new[1]), n, replace=False)
    # for i in to_drop:
    new[0][:,i] = 0
    new[1][i] = 0
    new[2][i,:] = 0
    model.set_weights(new)
    new_loss, new_acc = model.evaluate(x_test, y_test, verbose=0)
    score = (1 - (new_loss / loss)) + ((new_acc / acc) - 1)
    score1 = (1 - (new_loss / loss))
    score2 = ((new_acc / acc) - 1)
    if best_score < score1 or best_score < score2:
        best_score = score1
        best_model = copy.deepcopy(new)
    if best_score < score2:
        best_score = score2
        best_model = copy.deepcopy(new)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:24<00:00,  4.00it/s]


In [9]:
best_model[1]

array([ 0.16169311, -0.10345983,  0.00614873, -0.0794104 ,  0.16868322,
        0.        , -0.01383742,  0.16401708,  0.1587898 ,  0.18996462,
        0.08559503,  0.1642694 , -0.1717523 ,  0.0043246 ,  0.12378834,
        0.07347573,  0.16368593,  0.09373966,  0.10564376,  0.21958637,
       -0.11250514, -0.09608269,  0.13167442, -0.08325607, -0.07184245,
       -0.02978066, -0.08470738,  0.19127788,  0.19457266, -0.03510207,
       -0.02099682, -0.04862405, -0.14117543, -0.04790575, -0.07372855,
        0.10529654, -0.02385138,  0.01524916,  0.14234325,  0.06515182,
       -0.03439229, -0.00954995, -0.07570472, -0.15109669, -0.02583034,
       -0.213333  , -0.05014271,  0.1049145 ,  0.16233799,  0.0308647 ,
        0.02549643, -0.157727  , -0.00328106, -0.06673614,  0.10244828,
        0.13220301,  0.1977525 , -0.14430088,  0.16211155, -0.05398158,
       -0.16460484, -0.02360076,  0.04884092,  0.16300909, -0.02028583,
        0.13412923, -0.02871647,  0.19245613,  0.11810467,  0.05

In [10]:
model.set_weights(best_model)
loss, acc = model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0799 - accuracy: 0.9787


In [ ]:
best_score = 0
best_model = copy.deepcopy(old)
to_test = 25
for i in range(1, 65):
    temp_model, temp_score, nodes_removed = remove_random_nodes(acc, loss, old, i, to_test, x_train, y_train)
    if temp_score > best_score:
        best_model = temp_model
        best_score = temp_score
        print("Found new best model")

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

Found new best model


 40%|████████████████████████████████▊                                                 | 10/25 [00:11<00:17,  1.16s/it]

In [ ]:
best_model[1]

## Creating a restriced model

In [ ]:
n = 2
best_weights, _, nodes_removed = remove_random_nodes(acc, loss, old, n, 50, x_train, y_train)

new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - n)), np.zeros((best_weights[1].shape[0] - n)), np.zeros((best_weights[2].shape[0] - n, best_weights[2].shape[1])), best_weights[3]]

j = 0
for i in range(len(best_weights[1])):
    if i not in nodes_removed:
        new_weights[0][:, j] = best_weights[0][:, i]
        new_weights[1][j] = best_weights[1][i]
        new_weights[2][j, :] = best_weights[2][i, :]
        j = j + 1
    
new_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128 - n, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_model.set_weights(new_weights)

In [ ]:
def shrink_model(model, shrinkage_factor, x_train, y_train, size, to_test, v=0, remove='positive'):
    
    n = shrinkage_factor
    loss, acc = model.evaluate(x_train, y_train, verbose=2)
    old = model.get_weights()
    best_weights, _, nodes_removed = remove_random_nodes(acc, loss, old, n, to_test, x_train, y_train, v, remove)

    if nodes_removed.size:
        new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - n)), np.zeros((best_weights[1].shape[0] - n)), np.zeros((best_weights[2].shape[0] - n, best_weights[2].shape[1])), best_weights[3]]

        j = 0
        for i in range(len(best_weights[1])):
            if i not in nodes_removed:
                new_weights[0][:, j] = best_weights[0][:, i]
                new_weights[1][j] = best_weights[1][i]
                new_weights[2][j, :] = best_weights[2][i, :]
                j = j + 1

        new_model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(size - n, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
    else:
        print("Shrinking unsuccessful")
        return model, size
    
    new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    new_model.set_weights(new_weights)
    print("Shrinking successful")
    return new_model, size-n

In [ ]:
new_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
new_model.fit(x_train, y_train, epochs=3)

In [ ]:
new_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
size = 128
to_test = 25
for _ in range(4):
    model.fit(x_train, y_train, epochs=1)
    model, size = shrink_model(model, 2, x_train, y_train, size, to_test)
    print(len(model.get_weights()[1]))
model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test, y_test, verbose=2)

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
rep = 6

best_models = []
sizes = []
scores = []
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
start_weights = copy.deepcopy(model.get_weights())
model.fit(x_train, y_train, epochs=7)
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("#############################")
best_models += [model]
scores += [(loss, acc)]
sizes +=[128]
for i in range(1, 6):
    print(f"Starting to shrinking the model by {i}")
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.set_weights(start_weights)
    size = 128
    to_test = 15
    for _ in range(rep):
        model.fit(x_train, y_train, epochs=1)
        model, size = shrink_model(model, i, x_train, y_train, size, to_test, remove='always')
    model.fit(x_train, y_train, epochs=1)
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    print("#############################")
    best_models += [model]
    scores += [(loss, acc)]
    sizes +=[(len(model.get_weights()[1]))]

In [ ]:
print(scores)
sizes = [128, 122, 116, 110, 104, 98]
print(sizes)

In [ ]:
scores_plain = [scores[0]]
for i in range(1, len(scores)):
    print(f"Starting plain train of Dense size {sizes[i]}")
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(sizes[i], activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=7)
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    scores_plain += [(loss, acc)]
    print("###############################")

In [ ]:
best_models_p = []
sizes_p = []
scores_p = []
best_models_p += [model]
scores_p += [(loss, acc)]
sizes_p +=[128]
for i in range(1, 6):
    print(f"Starting to shrinking the model by {i}")
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.set_weights(start_weights)
    size = 128
    to_test = 15
    for _ in range(rep):
        model.fit(x_train, y_train, epochs=1)
        model, size = shrink_model(model, i, x_train, y_train, size, to_test, remove='positive')
    model.fit(x_train, y_train, epochs=1)
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    print("#############################")
    best_models += [model]
    scores_p += [(loss, acc)]
    sizes_p +=[(len(model.get_weights()[1]))]

In [ ]:
print(sizes_p)

In [ ]:
print(scores)
print("#############################")
print(scores_p)
print("#############################")
print(scores_plain)

In [ ]:
for i in range(len(scores)):
    print("Loss change:", (scores_plain[i][0] - scores[i][0])/scores_plain[i][0] *100, "--- Acc change:", -(scores_plain[i][1] - scores[i][1]) / scores_plain[i][1] * 100)

In [ ]:
to_remove_list = np.arange(1, 65)
num_rep = 10
loss_diff = np.zeros(num_rep)
acc_diff = np.zeros(num_rep)
loss_change = np.zeros(num_rep)
acc_change = np.zeros(num_rep)
nodes_removed_list = []
num_nodes_removed = np.zeros(num_rep)

In [ ]:
print(to_remove_list)

In [ ]:
for i in range(num_rep):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10)
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    
    n = np.random.choice(to_remove_list, 1)
    
    best_weights, _, nodes_removed = remove_random_nodes(acc, loss, model.get_weights(), n, 1, x_train, y_train, 0, remove='always')
    
    model.set_weights(best_weights)
    print(n)
    
    loss_new, acc_new = model.evaluate(x_test, y_test, verbose=2)
    
    loss_diff[i] = loss - loss_new
    acc_diff[i] = acc_new - acc
    loss_change[i] = loss_diff[i] / loss * 100
    acc_change[i] = acc_diff[i] / acc * 100
    num_nodes_removed[i] = n
    nodes_removed_list += [nodes_removed]

In [ ]:
for i in range(1, 65):
    print(f"{i} nodes removed")
    print("Loss changes:",loss_change[num_nodes_removed == i])
    print("Accuracy changes:",acc_change[num_nodes_removed == i])
    print("#########################")